## Import relevant packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Loading the data

In [3]:
mnist_dataset , mnist_info = tfds.load(name = 'mnist' , with_info = True , as_supervised = True , try_gcs=True)

## Extract train , validation and test dataset

In [5]:
mnist_train , mnist_test = mnist_dataset['train'] , mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples #gives num of training set samples
num_validation_samples = tf.cast(num_validation_samples , tf.int64) #makes sure its integar and not float

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast( num_test_samples, tf.int64)